In [ ]:
import numpy as np
from tqdm import tqdm

# TODO: var_rot_speed in rad/s^2, var_speed in mm/s^2, var_camera 
# TODO: get data from thymio and camera
# TODO: average speed and rotational speed
# TODO: if data is bad, might need to consider variance of left and right motor separately
# TODO: question for TA: ok if motor variance split equally in state and measurement variance and camera variance all only measurement variance?

var_motors = [75.72, 0.002692]  # variance for the motors [translational, rotational] from calibrations_thymio.ipynb
var_camera = [2.25, 2.25, 0.005]  # variance for the camera [x, y, theta] from calibrations_camera.ipynb

# Extended Kalman Filter
state contains position (x, y, rotation) and velocity (translational, rotational)
$ x = \begin{bmatrix} x & y & \theta & v & \omega \end{bmatrix} ^T$


non-linear discrete-time state-space model $ x_{k+1} = f(x_k, T_s) + w_k $:
$$
x_{k+1} = \begin{bmatrix}
            x_k + v \cdot cos(\theta_k) \cdot T_s \\
            y_k + v \cdot sin(\theta_k) \cdot T_s \\
            \theta_k + \omega \cdot T_s \\
            v_k \\
            \omega_k
            \end{bmatrix}
            + w_k
$$
with process noise (on states) w_k with zero mean and covariance Q. Here we use 
$$ 
Q = G \cdot \begin{bmatrix} 
                \sigma_{motor,trans} ^2 & 0 \\
                0 & \sigma_{motor,rot} ^2
                \end{bmatrix}
        \cdot G^T 
$$
with 
$$ G = \begin{bmatrix} 
        \frac{df}{dv} & \frac{df}{d\omega} 
        \end{bmatrix} 
    = \begin{bmatrix}
        cos(\theta_k) \cdot T_s & 0 \\
        sin(\theta_k) \cdot T_s & 0 \\
        0 & T_s \\
        1 & 0 \\
        0 & 1 
        \end{bmatrix}
$$

output model (measurements) $ y_k = H \cdot x_k + v_k $
with all states measured (first 3 from camera, last two from motor speeds), therefore:
$$ H = \begin{bmatrix}
        1 & 0 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 & 0 \\
        0 & 0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 0 & 1
        \end{bmatrix}
$$
and measurement noise $v_k$ with zero mean and covariance R. Here we assume, that all measurement noises are independent and therefore R is a diagonal matrix:
$$ R = \begin{bmatrix}
        \sigma_x ^2 & 0 & 0 & 0 & 0 \\
        0 & \sigma_y ^2 & 0 & 0 & 0\\
        0 & 0 & \sigma_\theta ^2  & 0 & 0 \\
        0 & 0 & 0 & \sigma_{motor,trans} ^2 & 0\\
        0 & 0 & 0 & 0 & \sigma_{motor,rot} ^2
        \end{bmatrix}
$$

In [ ]:
# how to call the ekf_filter function with rolling buffer

N = 50 # rolling buffer size
state_dim = 5  # state dimension
pos_dim = 3    # position dimension
speed_dim = 2  # speed dimension

# init buffers
x_est_buf = np.zeros((N, state_dim, 1))
P_est_buf = np.zeros((N, state_dim, state_dim))
speed_buf = np.zeros((N, speed_dim, 1))
pos_buf = np.zeros((N, pos_dim, 1))
# initial estimates
x_est_buf[0] = np.zeros((state_dim, 1))
P_est_buf[0] = 1000 * np.eye(state_dim)  # initial covariance
speed_buf[0] = np.zeros((speed_dim, 1)) # initial speed
pos_buf[0] = np.zeros((pos_dim, 1)) # initial position
#index for rolling buffer
idx = 0

# TODO: adapt!! this is only a copy from the exercise solution 8
for k in tqdm(range(55, len(thymio_data))):
    #update index
    idx = (idx + 1) % N
    #update speed and position buffers
    speed_buf[idx] = np.array([[avg_speed[k], avg_speed_rot[k]]])  # linear and rotational speed
    pos_buf[idx] = np.array([[camera_data[k]["x"], camera_data[k]["y"], camera_data[k]["theta"]]])  # position from camera
    
    #get previous estimates from buffer
    prev_idx = (idx - 1) % N
    x_est_buf[idx], P_est_buf[idx] = ekf_filter(speed_buf[idx], pos_buf[idx], x_est_buf[prev_idx], P_est_buf[prev_idx], var_motors, var_camera)